In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import datetime
%config InlineBackend.figure_format = 'retina'

import time
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import process

import re

/Users/sam/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data15 = pd.read_csv('./2015pcard_data (1).csv')
data16 = pd.read_csv('./2016pcard_data (1).csv')
data17 = pd.read_csv('./2017pcard_data (1).csv')
input16 = pd.read_csv('./techtrades_2016_data_input_CC (1).csv')
accounts = pd.read_csv('./techtrades_accounts (1).csv')
bu = pd.read_csv('./techtrades_bu (1).csv')

In [3]:
# This returns a df of tuples for each vendor (vendor match, match weight)
def get_fuzzy_matches(df, match_range):
    top10 = list(df[['vendor', 'amount']].groupby('vendor').sum().sort_values('amount', ascending=False).head(10).index)
    matches = []
    for vendor in tqdm(top10):
        match = process.extract(vendor, [i for i in df['vendor'].unique() if vendor != i], limit=match_range)
        matches.append(match)
    matches = pd.DataFrame(matches)
    matches['vendors'] = top10
    return matches[['vendors'] + [x for x in range(match_range)]]

In [4]:
matches17 = get_fuzzy_matches(data17, 5)
matches17.set_index('vendors', inplace=True)
matches17

100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


,0,1,2,3,4
vendors,,,,,
IBM,"(TIM HORTONS, 68)","(TIM HORTON'S, 68)","(TIM HORTONS 2601 QTH, 68)","(TIM HORTONS 1574 QTH, 68)","(AAROPORT LIMOUSINE SER, 60)"
CDW Direct,"(CDW DIRECT LLC, 95)","(SATCOM DIRECT INC, 86)","(DTV*DIRECTV HARDWARE, 72)","(DIRECTV, 71)","(STAPLES DIRECT, 71)"
"Apple, Inc.","(ABC DISPOSAL SYSTEMS INC, 86)","(AGS FOOTWEAR INC., 86)","(ALLIED SAFE & LOCK CO INC, 86)","(CATAYLST TAGS INC, 86)","(CROWN AWARDS INC, 86)"
Microsoft Corporation,"(DACO CORPORATION, 77)","(CHOSUN HOTEL CORPORATION, 76)","(DEMATIC CORPORATION, 75)","(QUILL CORPORATION, 75)","(EQUIPARTS CORPORATION, 71)"
CENTURYLINK,"(CenturyLink, 100)","(CENTRO, 75)","(INK T, 68)","(CENTURY CAFE, 61)","(Concur, 60)"
"SMART IMS, INC","(ABC DISPOSAL SYSTEMS INC, 86)","(ALLIED SAFE & LOCK CO INC, 86)","(DataArt Solutions, Inc, 86)","(ENGENIUS TECHNOLOGY INC, 86)","(FIELDWORK SEATTLE INC, 86)"
Amazon AWS,"(MARRIOTT SAN RAMON CA, 64)","(WB MASON, 64)","(A&W, 60)","(AMOENA, 60)","(WSBA, 60)"
"Oracle, Inc.","(ABC DISPOSAL SYSTEMS INC, 86)","(ALLIED SAFE & LOCK CO INC, 86)","(CATAYLST TAGS INC, 86)","(DATA SALES CO INC., 86)","(DataArt Solutions, Inc, 86)"
"DataArt Solutions, Inc","(AEROTEK INC., 86)","(AGUAMAN INC, 86)","(Apple, Inc., 86)","(BAUDVILLE INC., 86)","(CARSCO INC GGA, 86)"


In [5]:
matches16 = get_fuzzy_matches(data16, 5)
matches16.set_index('vendors', inplace=True)
matches16

100%|██████████| 10/10 [00:07<00:00,  1.16it/s]


,0,1,2,3,4
vendors,,,,,
IBM,"(TIM HORTONS, 68)","(TIM HORTON'S, 68)","(TIM HORTONS 1574 QTH, 68)","(TIM HORTONS 2601 QTH, 68)","(AAROPORT LIMOUSINE SER, 60)"
CDW Direct,"(CDW DIRECT LLC, 95)","(SATCOM DIRECT INC, 86)","(DTV*DIRECTV HARDWARE, 72)","(DIRECTV, 71)","(STAPLES DIRECT, 71)"
"Oracle, Inc.","(ABC DISPOSAL SYSTEMS INC, 86)","(ALLIED SAFE & LOCK CO INC, 86)","(CATAYLST TAGS INC, 86)","(DATA SALES CO INC., 86)","(DataArt Solutions, Inc, 86)"
"SALESFORCE.COM, INC.","(AEROTEK INC., 86)","(Agio Inc, 86)","(AGUAMAN INC, 86)","(Apple, Inc., 86)","(DOCKER INC, 86)"
"DataArt Solutions, Inc","(AEROTEK INC., 86)","(Agio Inc, 86)","(AGUAMAN INC, 86)","(Apple, Inc., 86)","(BAUDVILLE INC., 86)"
"Apple, Inc.","(ABC DISPOSAL SYSTEMS INC, 86)","(AGS FOOTWEAR INC., 86)","(ALLIED SAFE & LOCK CO INC, 86)","(CATAYLST TAGS INC, 86)","(CROWN AWARDS INC, 86)"
"SMART IMS, INC","(ABC DISPOSAL SYSTEMS INC, 86)","(Agio Inc, 86)","(ALLIED SAFE & LOCK CO INC, 86)","(DataArt Solutions, Inc, 86)","(DECISIONPOINT SYSTEMS INC, 86)"
Amazon AWS,"(MARRIOTT SAN RAMON CA, 64)","(WB MASON, 64)","(A&W, 60)","(AMOENA, 60)","(WSBA, 60)"
"Oxford Health Insurance, Inc","(AEROTEK INC., 86)","(Agio Inc, 86)","(AGS FOOTWEAR INC., 86)","(AGUAMAN INC, 86)","(Apple, Inc., 86)"


In [6]:
matches15 = get_fuzzy_matches(data15, 5)
matches15.set_index('vendors', inplace=True)
matches15

100%|██████████| 10/10 [02:35<00:00, 15.54s/it]


,0,1,2,3,4
vendors,,,,,
IBM,"(DOUBLETREE STES ANAHEIM, 72)","(H AND H KIM, 72)","(MANFROTTO DISTRIB, 72)","(PHO KIM, 72)","(RANJAD DISTIB, 72)"
"Apple, Inc.","(LE, 90)","(Apple, 90)","(003 Oxford Health Insurance, Inc MY ACCOUN, 86)","(004 Oxford Health Insurance, Inc IVR, 86)","(005 Oxford Health Insurance, Inc RECURRING, 86)"
CenturyLink,"(CENTURYLINK, 100)","(CenturyLink F/B, 95)","(EN, 90)","(SQ *QUALSERV CENTURYLINK., 90)","(ENT, 90)"
"Oracle, Inc.","(LE, 90)","(003 Oxford Health Insurance, Inc MY ACCOUN, 86)","(004 Oxford Health Insurance, Inc IVR, 86)","(005 Oxford Health Insurance, Inc RECURRING, 86)","(03 MCLENDON HARDWARE INC, 86)"
AVAYA BD USA,"(USA, 90)","(3RD GENERATION USA, 86)","(85C BAKERY CAFE USA, 86)","(Am Bd for Cert in O&P, 86)","(BD'S MONGOLIAN GRI, 86)"
LinkedIn Corporation,"(LinkedIn Corporation BOARD RM, 95)","(LINKEDIN, 90)","(INK, 90)","(LINKEDIN.COM, 86)","(O2 Corporation, 84)"
"SMART IMS, INC","(IMS, 90)","(003 Oxford Health Insurance, Inc MY ACCOUN, 86)","(004 Oxford Health Insurance, Inc IVR, 86)","(005 Oxford Health Insurance, Inc RECURRING, 86)","(03 MCLENDON HARDWARE INC, 86)"
Taboola Inc.,"(Taboola Inc. F&B, 95)","(Taboola Inc. F/B, 95)","(DAMONS Taboola Inc., 90)","(DOWNTOWN Taboola Inc. HOTE, 90)","(FOUR POINTS BY Taboola Inc., 90)"
OFFICE DEPOT,"(OFICE DEPOT, 96)","(OFFICE MAX/OFFICE DEPOT, 90)","(OFFICEMAX/OFFICE DEPOT, 90)","(DE, 90)","(1031 CENTRAL OFFICE, 86)"


In [8]:
# Creates dict {top 10 vendor names: [list of fuzzy matched vendor names]}
def filter_matches(df, threshold):
    filtered = {}
    for vendor in df.index:
        if any(np.array([x[1] for x in df.loc[vendor]]) >= threshold):
            filtered[vendor] = [vendor] + [x[0] for x in df.loc[vendor] if x[1] >= threshold]
    return filtered

In [9]:
match_dict_17 = filter_matches(matches17, 90)
match_dict_17

{'CDW Direct': ['CDW Direct', 'CDW DIRECT LLC'],
 'CENTURYLINK': ['CENTURYLINK', 'CenturyLink']}

In [10]:
# This misses several OFFICEMAX/OFFICEDEPOTS from the fuzzy matches
match_dict_16 = filter_matches(matches16, 90)
match_dict_16

{'CDW Direct': ['CDW Direct', 'CDW DIRECT LLC'],
 'OFFICE DEPOT': ['OFFICE DEPOT', 'OFFICEMAX/OFFICE DEPOT']}

In [11]:
# This misses too many non-matching vendors, and not getting all matches for Taboola.
match_dict_15 = filter_matches(matches15, 90)
match_dict_15

{'Apple, Inc.': ['Apple, Inc.', 'LE', 'Apple'],
 'CenturyLink': ['CenturyLink',
  'CENTURYLINK',
  'CenturyLink F/B',
  'EN',
  'SQ *QUALSERV CENTURYLINK.',
  'ENT'],
 'Oracle, Inc.': ['Oracle, Inc.', 'LE'],
 'AVAYA BD USA': ['AVAYA BD USA', 'USA'],
 'LinkedIn Corporation': ['LinkedIn Corporation',
  'LinkedIn Corporation BOARD RM',
  'LINKEDIN',
  'INK'],
 'SMART IMS, INC': ['SMART IMS, INC', 'IMS'],
 'Taboola Inc.': ['Taboola Inc.',
  'Taboola Inc. F&B',
  'Taboola Inc. F/B',
  'DAMONS Taboola Inc.',
  'DOWNTOWN Taboola Inc. HOTE',
  'FOUR POINTS BY Taboola Inc.'],
 'OFFICE DEPOT': ['OFFICE DEPOT',
  'OFICE DEPOT',
  'OFFICE MAX/OFFICE DEPOT',
  'OFFICEMAX/OFFICE DEPOT',
  'DE'],
 'STAPLES': ['STAPLES',
  'STAPLES.CA',
  'STAPLS',
  'S',
  'STAPLES ACCOUN',
  'STAPLES DIRECT']}

In [12]:
match_dict = dict(match_dict_17, **match_dict_16)
match_dict = dict(match_dict, **match_dict_15)
match_dict

{'CDW Direct': ['CDW Direct', 'CDW DIRECT LLC'],
 'CENTURYLINK': ['CENTURYLINK', 'CenturyLink'],
 'OFFICE DEPOT': ['OFFICE DEPOT',
  'OFICE DEPOT',
  'OFFICE MAX/OFFICE DEPOT',
  'OFFICEMAX/OFFICE DEPOT',
  'DE'],
 'Apple, Inc.': ['Apple, Inc.', 'LE', 'Apple'],
 'CenturyLink': ['CenturyLink',
  'CENTURYLINK',
  'CenturyLink F/B',
  'EN',
  'SQ *QUALSERV CENTURYLINK.',
  'ENT'],
 'Oracle, Inc.': ['Oracle, Inc.', 'LE'],
 'AVAYA BD USA': ['AVAYA BD USA', 'USA'],
 'LinkedIn Corporation': ['LinkedIn Corporation',
  'LinkedIn Corporation BOARD RM',
  'LINKEDIN',
  'INK'],
 'SMART IMS, INC': ['SMART IMS, INC', 'IMS'],
 'Taboola Inc.': ['Taboola Inc.',
  'Taboola Inc. F&B',
  'Taboola Inc. F/B',
  'DAMONS Taboola Inc.',
  'DOWNTOWN Taboola Inc. HOTE',
  'FOUR POINTS BY Taboola Inc.'],
 'STAPLES': ['STAPLES',
  'STAPLES.CA',
  'STAPLS',
  'S',
  'STAPLES ACCOUN',
  'STAPLES DIRECT']}

In [15]:
# Manually remove non-matches
match_dict['OFFICE DEPOT'] = [x for x in match_dict['OFFICE DEPOT'] if x not in ['DE']]
match_dict['Apple, Inc.'] = [x for x in match_dict['Apple, Inc.'] if x not in ['LE']]
match_dict['CenturyLink'] = [x for x in match_dict['CenturyLink'] if x not in ['EN', 'ENT']]
match_dict['LinkedIn Corporation'] = [x for x in match_dict['LinkedIn Corporation'] if x not in ['INK']]
match_dict['Oracle, Inc.'] = [x for x in match_dict['Oracle, Inc.'] if x not in ['LE']]
match_dict['STAPLES'] = [x for x in match_dict['STAPLES'] if x not in ['S']]

match_dict

{'CDW Direct': ['CDW Direct', 'CDW DIRECT LLC'],
 'CENTURYLINK': ['CENTURYLINK', 'CenturyLink'],
 'OFFICE DEPOT': ['OFFICE DEPOT',
  'OFICE DEPOT',
  'OFFICE MAX/OFFICE DEPOT',
  'OFFICEMAX/OFFICE DEPOT'],
 'Apple, Inc.': ['Apple, Inc.', 'Apple'],
 'CenturyLink': ['CenturyLink',
  'CENTURYLINK',
  'CenturyLink F/B',
  'SQ *QUALSERV CENTURYLINK.'],
 'Oracle, Inc.': ['Oracle, Inc.'],
 'AVAYA BD USA': ['AVAYA BD USA', 'USA'],
 'LinkedIn Corporation': ['LinkedIn Corporation',
  'LinkedIn Corporation BOARD RM',
  'LINKEDIN'],
 'SMART IMS, INC': ['SMART IMS, INC', 'IMS'],
 'Taboola Inc.': ['Taboola Inc.',
  'Taboola Inc. F&B',
  'Taboola Inc. F/B',
  'DAMONS Taboola Inc.',
  'DOWNTOWN Taboola Inc. HOTE',
  'FOUR POINTS BY Taboola Inc.'],
 'STAPLES': ['STAPLES',
  'STAPLES.CA',
  'STAPLS',
  'STAPLES ACCOUN',
  'STAPLES DIRECT']}

In [20]:
# Manually finding officemax/officedepots
offices = []
for x in list(data16['vendor'].unique()):
    try:
        if re.findall(r'OFFICEMAX|OFFICEDEPOT|Office Max|Office Depot', x):
            offices.append(x)
    except:
        pass

match_dict['OFFICE DEPOT'].extend(offices)
match_dict['OFFICE DEPOT']

['OFFICE DEPOT',
 'OFICE DEPOT',
 'OFFICE MAX/OFFICE DEPOT',
 'OFFICEMAX/OFFICE DEPOT',
 'Office Max',
 'Office Max ON BOARD CA',
 'OFFICEMAX/OFFICEDEPOT',
 'OFFICEMAX/OFFICEDEPOT6161',
 'OFFICEMAX/OFFICEDEPOT6169',
 'OFFICEMAX/OFFICEDEPOT6515',
 'OFFICEMAX/OFFICEDEPOT6840',
 'OFFICEMAX/OFFICEDEPT']

In [21]:
# def remove_small_matches(match_dict, threshold):
#     for main_vendor in match_dict:
#         for vendor in match_dict[main_vendor]:
#             if len(vendor) < len(main_vendor)*threshold:
#                 print('removing', main_vendor, vendor)
#                 match_dict[main_vendor] = [x for x in match_dict[main_vendor] if x is not vendor]
#     return match_dict
# remove_small_matches(mat15, .3)

In [29]:
def regex_vendor_matches(matches_df, df):
    vendor_matches = {}
    for vendor in matches_df.index:
        vendor_matches[vendor] = []
        for x in list(df['vendor'].unique()):
            if re.findall(r'{}'.format(str(vendor).lower()), str(x).lower()):
                vendor_matches[vendor].append(x)
    return vendor_matches

In [30]:
r_matches_15 = regex_vendor_matches(matches15, data15)
r_matches_15

{'IBM': ['IBM'],
 'Apple, Inc.': ['Apple, Inc.'],
 'CenturyLink': ['CenturyLink', 'CenturyLink-INDPL F&B'],
 'Oracle, Inc.': ['Oracle, Inc.'],
 'AVAYA BD USA': ['AVAYA BD USA'],
 'LinkedIn Corporation': ['LinkedIn Corporation'],
 'SMART IMS, INC': ['SMART IMS, INC'],
 'Taboola Inc.': ['Taboola Inc.',
  'FOUR POINTS BY Taboola Inc. C',
  'FOUR POINTS BY Taboola Inc. H',
  'FOUR POINTS Taboola Inc. BREN',
  'FOUR POINTS Taboola Inc. SJC',
  'PMSTaboola Inc.SHARLOTTE',
  'RAIN 903 Taboola Inc. BALT',
  'Taboola Inc. AGOURA HILLS LOD',
  'Taboola Inc. ANCHORAGE F&B',
  'Taboola Inc. ARLINGTN F&B',
  'Taboola Inc. ARLINGTON',
  'Taboola Inc. BLOOMINGTON HOTE',
  'Taboola Inc. BWI WASH AIRPORT',
  'Taboola Inc. CALIFORNIAN',
  'Taboola Inc. CAVALIER',
  'Taboola Inc. CITY CENTER',
  'Taboola Inc. CITY CNTR FB',
  'Taboola Inc. CLAYTON PLAZA F',
  'Taboola Inc. COLUMBIA F&B',
  'Taboola Inc. CONV CTR DINING',
  'Taboola Inc. DELFINA REST',
  'Taboola Inc. DENVER',
  'Taboola Inc. DENVER DININ

In [31]:
match_dict['Taboola Inc.'].extend(r_matches_15['Taboola Inc.'])

In [32]:
match_dict

{'CDW Direct': ['CDW Direct', 'CDW DIRECT LLC'],
 'CENTURYLINK': ['CENTURYLINK', 'CenturyLink'],
 'OFFICE DEPOT': ['OFFICE DEPOT',
  'OFICE DEPOT',
  'OFFICE MAX/OFFICE DEPOT',
  'OFFICEMAX/OFFICE DEPOT',
  'Office Max',
  'Office Max ON BOARD CA',
  'OFFICEMAX/OFFICEDEPOT',
  'OFFICEMAX/OFFICEDEPOT6161',
  'OFFICEMAX/OFFICEDEPOT6169',
  'OFFICEMAX/OFFICEDEPOT6515',
  'OFFICEMAX/OFFICEDEPOT6840',
  'OFFICEMAX/OFFICEDEPT'],
 'Apple, Inc.': ['Apple, Inc.', 'Apple'],
 'CenturyLink': ['CenturyLink',
  'CENTURYLINK',
  'CenturyLink F/B',
  'SQ *QUALSERV CENTURYLINK.'],
 'Oracle, Inc.': ['Oracle, Inc.'],
 'AVAYA BD USA': ['AVAYA BD USA', 'USA'],
 'LinkedIn Corporation': ['LinkedIn Corporation',
  'LinkedIn Corporation BOARD RM',
  'LINKEDIN'],
 'SMART IMS, INC': ['SMART IMS, INC', 'IMS'],
 'Taboola Inc.': ['Taboola Inc.',
  'Taboola Inc. F&B',
  'Taboola Inc. F/B',
  'DAMONS Taboola Inc.',
  'DOWNTOWN Taboola Inc. HOTE',
  'FOUR POINTS BY Taboola Inc.',
  'Taboola Inc.',
  'FOUR POINTS BY Ta

In [33]:
# Doesn't return any new matches
r_matches_16 = regex_vendor_matches(matches16, data16)
r_matches_16

{'IBM': ['IBM'],
 'CDW Direct': ['CDW Direct'],
 'Oracle, Inc.': ['Oracle, Inc.'],
 'SALESFORCE.COM, INC.': ['SALESFORCE.COM, INC.'],
 'DataArt Solutions, Inc': ['DataArt Solutions, Inc'],
 'Apple, Inc.': ['Apple, Inc.'],
 'SMART IMS, INC': ['SMART IMS, INC'],
 'Amazon AWS': ['Amazon AWS'],
 'Oxford Health Insurance, Inc': ['Oxford Health Insurance, Inc'],
 'OFFICE DEPOT': ['OFFICE DEPOT']}

In [34]:
# Doesn't return any new matches
r_matches_17 = regex_vendor_matches(matches17, data17)
r_matches_17

{'IBM': ['IBM'],
 'CDW Direct': ['CDW Direct'],
 'Apple, Inc.': ['Apple, Inc.'],
 'Microsoft Corporation': ['Microsoft Corporation'],
 'CENTURYLINK': ['CenturyLink'],
 'SMART IMS, INC': ['SMART IMS, INC'],
 'Amazon AWS': ['Amazon AWS'],
 'Oracle, Inc.': ['Oracle, Inc.'],
 'DataArt Solutions, Inc': ['DataArt Solutions, Inc'],
 'SALESFORCE.COM, INC.': ['SALESFORCE.COM, INC.']}

In [35]:
def remap_vendors(matches_dict, df):
    for x in matches_dict:
        df.loc[df['vendor'].isin(matches_dict[x]), 'vendor'] = x
    return round(df[['vendor', 'amount']].groupby('vendor').sum().sort_values('amount', ascending=False).head(10), 2)

In [36]:
remap_vendors(match_dict, data17)

,amount
vendor,
IBM,4559476.00
CDW Direct,3455052.91
"Apple, Inc.",2408585.51
CenturyLink,2243072.22
Microsoft Corporation,2178931.11
"SMART IMS, INC",1729184.70
Amazon AWS,1668552.30
"Oracle, Inc.",1529637.48
"DataArt Solutions, Inc",1270111.87


In [38]:
remap_vendors(match_dict, data16)

,amount
vendor,
IBM,6773290.85
CDW Direct,5841067.58
"Oracle, Inc.",3892199.00
"SALESFORCE.COM, INC.",3884109.09
"DataArt Solutions, Inc",2673467.57
"Apple, Inc.",2611532.34
"SMART IMS, INC",2374319.86
Amazon AWS,2232727.00
"Oxford Health Insurance, Inc",2049243.62


In [40]:
remap_vendors(match_dict, data15)

,amount
vendor,
IBM,10543266.07
CenturyLink,5356251.57
"Apple, Inc.",5180686.48
"Oracle, Inc.",4144153.00
Taboola Inc.,2292496.52
AVAYA BD USA,2264540.06
LinkedIn Corporation,2222716.33
"SMART IMS, INC",2138345.99
OFFICE DEPOT,1796525.38
